# Préparation et nettoyage des données DVF

**Objectif** : préparer un jeu de données propre pour l'analyse des **locaux commerciaux / boutiques** à **Paris** (arrondissements, prix/m², année, géolocalisation).



## Fichier à télécharger :

Dataset DVF (téléchargement csv.gz DVF2020-2024):
https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-geolocalisees

In [1]:
import pandas as pd
import numpy as np
import os

#### Lecture du fichier dvf.csv

On définit en premier lieu les **colonnes utiles** qui vont nous servir pour l'extraction et le nettoyage des données, dans le but d'**optimiser la lecture du fichier csv** qui est assez lourd (*+3Go*). Toujours dans le même objectif, la lecture se fait par **chunk** de 100000 lignes. Durant le processus, on réalise :
- La suppression des valeurs NaN
- La modification du type de la colonne 'date_mutation' en datetime
- La reduction des données au type "Local industriel, commercial ou assimilé"
- La filtration pour conserver uniquement les biens sur Paris

In [2]:
# Define columns to read upfront
colonnes_a_lire = [
    'date_mutation',
    'valeur_fonciere',
    'code_postal',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'longitude',
    'latitude',
    'adresse_nom_voie',
    'nom_commune'
]

# Process CSV in chunks
chunk_size = 100000
filtered_chunks = []

for i, chunk in enumerate(pd.read_csv('dvf.csv', 
                                       usecols=colonnes_a_lire,
                                       chunksize=chunk_size,
                                       low_memory=False)):
    
    # Drop NaN values and create explicit copy
    chunk = chunk.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 
                                  'nombre_pieces_principales',
                                  'latitude', 'longitude']).copy()
    
    # Convert date
    chunk['date_mutation'] = pd.to_datetime(chunk['date_mutation'], errors='coerce')
    
    # Filter by type_local
    chunk = chunk[chunk['type_local'] == 'Local industriel. commercial ou assimilé'].copy()
    
    # Filter Paris postal codes
    chunk = chunk[chunk['code_postal'].astype(str).str.startswith('75')].copy()
    
    if len(chunk) > 0:
        filtered_chunks.append(chunk)

# Combine all filtered chunks
df = pd.concat(filtered_chunks, ignore_index=True)

##### On vérifie si les colonnes restantes contiennent des valeurs NaN

In [3]:
df.isna().sum()

date_mutation                0
valeur_fonciere              0
adresse_nom_voie             0
code_postal                  0
nom_commune                  0
type_local                   0
surface_reelle_bati          0
nombre_pieces_principales    0
longitude                    0
latitude                     0
dtype: int64

##### Puis on retire du dataframe la colonne "type_local"

In [4]:
df = df.drop('type_local', axis=1)

##### Création d'une feature prix m²

In [5]:
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

##### On supprime les valeurs aberrantes au m²

In [6]:
df = df[(df['prix_m2'] > 500) & (df['prix_m2'] < 50000)]

##### Extraction de l'arrondissement (normalisation de code_postal)

In [7]:
df['code_postal'] = df['code_postal'].astype(int).astype(str).str.zfill(5)
df['arrondissement'] = pd.to_numeric(df['code_postal'].str[-2:], errors='coerce')
df.loc[~df['arrondissement'].between(1,20), 'arrondissement'] = np.nan
df['arrondissement'] = df['arrondissement'].dropna().astype(int)

##### On supprime la colonne code_postal (c'est toujours Paris)

In [8]:
df = df.drop('code_postal', axis=1)

##### Certains noms de communes ne sont pas Paris, il faut donc les supprimer

In [9]:
df = df[df['nom_commune'].str.startswith('Paris')]
df.shape

(17586, 10)

# Nettoyage BDD Loyer

In [10]:
loyer_df = pd.read_csv("loyers.csv", sep=';', encoding='utf-8')

##### On supprime les colonnes inutiles

In [11]:
colonnes_a_supprimer = ["Secteurs géographiques", "Numéro du quartier", "Ville","geo_shape"]
loyer_df = loyer_df.drop(columns=colonnes_a_supprimer, errors='ignore')

##### Le dataframe ne contient pas de valeurs NaN ou null, ni de doublons

##### Nettoyage des coordonnées GPS (format "lat, lon")

In [12]:
loyer_df[['lat', 'lon']] = loyer_df['geo_point_2d'].str.split(',', expand=True)
loyer_df['lat'] = loyer_df['lat'].astype(float)
loyer_df['lon'] = loyer_df['lon'].astype(float)

##### Suppression de la colonne 'geo_point_2d'

In [13]:
loyer_df = loyer_df.drop('geo_point_2d', axis=1)

In [14]:
df.head()

,date_mutation,valeur_fonciere,adresse_nom_voie,nom_commune,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,prix_m2,arrondissement
26,2020-01-06,878378.0,BD DE LA MADELEINE,Paris 1er Arrondissement,43.0,0.0,2.326882,48.869331,20427.395349,1.0
27,2020-01-13,475000.0,RUE DE WASHINGTON,Paris 8e Arrondissement,50.0,0.0,2.301658,48.872630,9500.000000,8.0
28,2020-01-03,800000.0,BD DE SEBASTOPOL,Paris 3e Arrondissement,125.0,0.0,2.351054,48.863055,6400.000000,3.0
29,2020-01-06,710000.0,RUE SAINT HONORE,Paris 1er Arrondissement,30.0,0.0,2.326348,48.866939,23666.666667,1.0
31,2020-01-09,17400000.0,RUE DU FAUBOURG SAINT HONORE,Paris 8e Arrondissement,387.0,0.0,2.309755,48.873496,44961.240310,8.0


# Jointure du dataframe des deux dataframes

In [15]:
from scipy.spatial import cKDTree

In [16]:
coords_quartiers = loyer_df[['lat', 'lon']].values
tree = cKDTree(coords_quartiers)

In [17]:
coords_dvf = df[['latitude', 'longitude']].values

In [18]:
distances, indices = tree.query(coords_dvf)

#### Ajouter les colonnes au DataFrame DVF

In [19]:
df['quartier'] = loyer_df.iloc[indices]['Nom du quartier'].values
df['numero_insee_quartier'] = loyer_df.iloc[indices]['Numéro INSEE du quartier'].values
df['loyer_ref_m2'] = loyer_df.iloc[indices]['Loyers de référence'].values
df['loyer_majore_m2'] = loyer_df.iloc[indices]['Loyers de référence majorés'].values
df['loyer_minore_m2'] = loyer_df.iloc[indices]['Loyers de référence minorés'].values
df['distance_quartier_m'] = distances * 111000  # Conversion approximative en mètres

#### Checking s'il y a pas des lignes ou le quartier est loin de la réalité

In [20]:
df = df[df['distance_quartier_m'] <= 1000]

In [21]:
df.head()

,date_mutation,valeur_fonciere,adresse_nom_voie,nom_commune,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,prix_m2,arrondissement,quartier,numero_insee_quartier,loyer_ref_m2,loyer_majore_m2,loyer_minore_m2,distance_quartier_m
26,2020-01-06,878378.0,BD DE LA MADELEINE,Paris 1er Arrondissement,43.0,0.0,2.326882,48.869331,20427.395349,1.0,Place-Vendôme,7510104,29.3,35.16,20.51,318.554490
27,2020-01-13,475000.0,RUE DE WASHINGTON,Paris 8e Arrondissement,50.0,0.0,2.301658,48.872630,9500.000000,8.0,Faubourg-du-Roule,7510830,22.8,27.40,16.00,320.218020
28,2020-01-03,800000.0,BD DE SEBASTOPOL,Paris 3e Arrondissement,125.0,0.0,2.351054,48.863055,6400.000000,3.0,Sainte-Avoie,7510312,31.8,38.16,22.26,425.130070
29,2020-01-06,710000.0,RUE SAINT HONORE,Paris 1er Arrondissement,30.0,0.0,2.326348,48.866939,23666.666667,1.0,Place-Vendôme,7510104,29.3,35.16,20.51,248.094155
31,2020-01-09,17400000.0,RUE DU FAUBOURG SAINT HONORE,Paris 8e Arrondissement,387.0,0.0,2.309755,48.873496,44961.240310,8.0,Faubourg-du-Roule,7510830,22.8,27.40,16.00,629.632252


In [22]:
df['nombre_pieces_principales'].value_counts()

0.0    16486
Name: nombre_pieces_principales, dtype: int64

#### Création du fichier final dvf_loyers.csv

In [23]:
df.to_csv("dvf_loyers.csv", sep=";", encoding="utf-8", index=False)
df.shape

(16486, 16)

### Ouverture et préparation du fichier Commerces

In [31]:
import pandas as pd

# 1️⃣ Chargement du fichier des commerces
# (adapter le chemin selon ton environnement)
commerce = pd.read_csv("commerce.csv", sep=';', encoding_errors='replace')

# 2️⃣ Sélection des colonnes utiles
commerce = commerce[['X', 'Y', 'type']].copy()

# 3️⃣ Renommage clair des colonnes
commerce.rename(columns={'X': 'longitude', 'Y': 'latitude', 'type': 'categorie_commerce'}, inplace=True)

# 4️⃣ Nettoyage des données
# Suppression des lignes avec valeurs manquantes
commerce = commerce.dropna(subset=['longitude', 'latitude', 'categorie_commerce'])

# Suppression des doublons (même point + même catégorie)
commerce = commerce.drop_duplicates(subset=['longitude', 'latitude', 'categorie_commerce'])

# Filtrage des coordonnées aberrantes
# À Paris : latitude entre 48.80 et 48.90, longitude entre 2.25 et 2.42
commerce = commerce[
    (commerce['latitude'].between(48.80, 48.90)) &
    (commerce['longitude'].between(2.25, 2.42))
]

# Suppression des types génériques ou vides
commerce = commerce[commerce['categorie_commerce'].str.strip() != '']

# (Optionnel) Nettoyage des majuscules / minuscules
commerce['categorie_commerce'] = commerce['categorie_commerce'].str.strip().str.lower()

# 5️⃣ Aperçu final
print("✅ Données nettoyées :", commerce.shape)
print(commerce.head())




✅ Données nettoyées : (55890, 3)
       longitude   latitude categorie_commerce
46133   2.419019  48.846524           optician
46350   2.400633  48.885848         restaurant
46362   2.414612  48.879459            clothes
46509   2.393475  48.815530             bakery
46821   2.373102  48.817713         restaurant


### Création du CSV commerce_nettoye

In [32]:
commerce.to_csv("commerce_nettoye.csv", sep=";", encoding="utf-8", index=False)